In [1]:
import pandas as pd #the csv reading library
import matplotlib.pyplot as plt #the ploting library
import glob #to get file names
import numpy as np
from pandas import Series
import seaborn as sns
from matplotlib.pyplot import figure
import cmocean.cm as cmo
import seaborn as sns

from pylab import rcParams
from numpy import diff
from scipy import interpolate
import scipy.stats as stats

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

import matplotlib.gridspec as gridspec

import cmocean
import cmocean.cm as cmo
import seaborn as sns

from scipy.interpolate import interp1d

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [2]:
df = pd.read_csv('../Dissertation_projects/Data/CDOM_FAKE_test.csv', sep=',', index_col=0)
df['code'] = 'code'
df.loc[df['acdom_conc'] < 0, 'acdom_conc'] = 0
df

,acdom_conc,porosity,Depth,code
Sample_ID,,,,
214341,0.000000,0.810266,0.25,code
214341,0.000000,0.802089,0.75,code
214341,0.000000,0.801479,1.25,code
214343,8.911852,0.770604,1.75,code
214343,9.020099,0.738277,2.25,code
214343,9.053986,0.685740,2.75,code
214344,61.969312,0.570500,3.25,code
214344,61.716526,0.478391,3.75,code
214344,62.064445,0.464296,4.25,code


In [3]:
interpolated_data = []

codes_list = df['code'].unique()

for code in codes_list:
    data_iron = df[df['code'] == code]
    data_por = df[df['code'] == code]
    depth_fe_max = data_iron['Depth'].max()
    depth_fe_min = data_iron['Depth'].min()
    depth_po_max = data_por['Depth'].max()
    depth_po_min = data_por['Depth'].min()
    
    data_iron_2 = data_iron
    data_iron = data_iron.loc[data_iron['Depth'] >= depth_po_min]
    data_iron = data_iron.loc[data_iron['Depth'] <= depth_po_max] 
    
    depth = data_iron['Depth']
    depth = np.asarray(depth, dtype=float).ravel()
    fe2 = data_iron['acdom_conc']
    fe2 = np.asarray(fe2, dtype=float).ravel()
    
    depth_p = data_por['Depth']
    depth_p = np.asarray(depth_p, dtype=float).ravel()
    porosity = data_por['porosity']
    porosity = np.asarray(porosity, dtype=float).ravel()
    
    valid_indices = np.isfinite(porosity)
    porosity = porosity[valid_indices]
    depth_p = depth_p[valid_indices]
    
    cs = interp1d(depth_p, porosity)
    por_interpolated = cs(depth)
    
    interpolated_data.append(pd.DataFrame({'por_interpolated': por_interpolated,
                                           'Depth': depth,
                                           'CDOM_umolL': fe2,
                                           'code': code
                                          }))

interpolated_df = pd.concat(interpolated_data, ignore_index=True)
interpolated_df.dropna(subset=['Depth'], inplace=True)
interpolated_df

df_cal = interpolated_df
df_cal['sed_density_Gpercm3'] = 2.6

#fe uM to M 
df_cal['cdom_mol_per_m3'] =  df_cal['CDOM_umolL'] / 1000000 * 1000

df_cal['cdom_mol_per_m3_vol'] =  df_cal['cdom_mol_per_m3'] * df_cal['por_interpolated']

df_cal['depth_m'] = df_cal['Depth'] / 100

df_cal

,por_interpolated,Depth,CDOM_umolL,code,sed_density_Gpercm3,cdom_mol_per_m3,cdom_mol_per_m3_vol,depth_m
0,0.810266,0.25,0.000000,code,2.6,0.000000,0.000000,0.0025
1,0.802089,0.75,0.000000,code,2.6,0.000000,0.000000,0.0075
2,0.801479,1.25,0.000000,code,2.6,0.000000,0.000000,0.0125
3,0.770604,1.75,8.911852,code,2.6,0.008912,0.006868,0.0175
4,0.738277,2.25,9.020099,code,2.6,0.009020,0.006659,0.0225
5,0.685740,2.75,9.053986,code,2.6,0.009054,0.006209,0.0275
6,0.570500,3.25,61.969312,code,2.6,0.061969,0.035353,0.0325
7,0.478391,3.75,61.716526,code,2.6,0.061717,0.029525,0.0375
8,0.464296,4.25,62.064445,code,2.6,0.062064,0.028816,0.0425
9,0.446018,4.75,82.661083,code,2.6,0.082661,0.036868,0.0475


In [4]:
Frames = []

for code in codes_list: 
    df_play = df_cal[df_cal['code'] == code] 

    # Create a row with NaN values
    new_row = pd.DataFrame([np.nan] * len(df_play.columns)).T

    # Set column names for the new row
    new_row.columns = df_play.columns
    new_row['depth_m'] = 0
    new_row['cdom_mol_per_m3_vol'] = 0 

    # Append the new row to the DataFrame
    df_play = pd.concat([pd.DataFrame(new_row), df_play], ignore_index=True)

    ds_fet = df_play.dropna(subset=['depth_m', 'cdom_mol_per_m3_vol'])


    # Define the depth range you want to integrate over
    start_depth = 0
    stop_depth = .56

    # Interpolating concentration at depth 5 using linear interpolation
    fet_at_depth_5 = np.interp(stop_depth, ds_fet['depth_m'], ds_fet['cdom_mol_per_m3_vol'])

    #add he depth 5 data  
    new_row.columns = ds_fet.columns
    new_row['depth_m'] = stop_depth
    new_row['cdom_mol_per_m3_vol'] = fet_at_depth_5


    # Selecting data within the specified depth range
    selected_data_fet = ds_fet[(ds_fet['depth_m'] >= start_depth) & (ds_fet['depth_m'] <= stop_depth)]
    selected_data_fet = pd.concat([selected_data_fet, new_row])
    
    # Calculating the integral using the trapezoidal rule
    integral_fet = np.trapz(selected_data_fet['cdom_mol_per_m3_vol'], selected_data_fet['depth_m']) / stop_depth

    dic = {'code':[code],
           'cdom_mol_per_m3_0_to_5cm':[integral_fet]}
        
    slopes = pd.DataFrame(dic)
    slopes = slopes.set_index('code', drop=True)
    Frames.append(slopes)
    
stocks_pw_fe_5 = pd.concat(Frames, axis=0, ignore_index=False)

stocks_pw_fe_5['per'] = stocks_pw_fe_5['cdom_mol_per_m3_0_to_5cm']/0.0287
stocks_pw_fe_5

,cdom_mol_per_m3_0_to_5cm,per
code,,
code,0.028627,0.997461
